# Searching for Sentinel-2 Scenes

Quick demo of `maji.search` — finds Sentinel-2 L2A imagery from the free CDSE STAC catalog.

> For implementation details, see `01_research_search.ipynb`.

In [ ]:
from maji import search_scenes, select_best_scenes, DOWNLOAD_BANDS

print(f"Bands: {DOWNLOAD_BANDS}")

## Define Search Area

In [ ]:
BBOX = (40.6, -2.6, 41.6, -1.6)  # Faza region, Kenya
START, END = "2025-10-01", "2025-12-31"
MAX_CLOUD = 40.0

In [ ]:
scenes = search_scenes(bbox=BBOX, start=START, end=END, max_cloud=MAX_CLOUD, verbose=True)

print(f"\nFound {len(scenes)} scenes across {scenes['mgrs_tile'].nunique()} tiles")
scenes[["scene_id", "mgrs_tile", "datetime", "cloud_cover"]].head()

In [ ]:
best = select_best_scenes(scenes, strategy="least_cloudy", verbose=True)

print(f"\nSelected {len(best)} scene(s) — one per tile")
best[["scene_id", "mgrs_tile", "datetime", "cloud_cover"]]

In [ ]:
from pathlib import Path

out_dir = Path("../DATA/search_results")
out_dir.mkdir(parents=True, exist_ok=True)
best.to_parquet(out_dir / "best_scenes.parquet")
print(f"Saved {len(best)} scene(s) to {out_dir / 'best_scenes.parquet'}")

## Next Steps

The selected scenes have been saved to `DATA/search_results/best_scenes.parquet`. The download notebook loads this file:

```python
import geopandas as gpd
best = gpd.read_parquet("../DATA/search_results/best_scenes.parquet")
```

The `assets` column contains S3 paths for each band. Pass these to `download_tile()`:

```python
from maji import download_tile, create_s3_session

session = create_s3_session(access_key, secret_key)
for _, row in best.iterrows():
    download_tile(row["assets"], row["mgrs_tile"], row["datetime"].strftime("%Y-%m-%d"), data_dir)
```

See `05_maji_download.ipynb` for the complete download workflow.